<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/preprocessing1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download datasets
! wget 'https://raw.githubusercontent.com/wuyifan18/DeepLog/master/data/hdfs_train'
! wget 'https://raw.githubusercontent.com/wuyifan18/DeepLog/master/data/hdfs_test_normal'
! wget 'https://raw.githubusercontent.com/wuyifan18/DeepLog/master/data/hdfs_test_abnormal'

--2023-07-11 09:11:21--  https://raw.githubusercontent.com/wuyifan18/DeepLog/master/data/hdfs_train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 253021 (247K) [text/plain]
Saving to: ‘hdfs_train’

hdfs_train          100%[===================>] 247.09K  --.-KB/s    in 0.03s   

2023-07-11 09:11:21 (7.74 MB/s) - ‘hdfs_train’ saved [253021/253021]

--2023-07-11 09:11:22--  https://raw.githubusercontent.com/wuyifan18/DeepLog/master/data/hdfs_test_normal
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28730917 (27M) [text/plain]
Sav

In [2]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
import argparse
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
# count rows of the hdfs_train

with open('/content/hdfs_train', "r") as file:
    totaln=0
    for line in file:
        totaln += 1

print('There are a total of {} lines'.format(totaln))

There are a total of 4855 lines


important point:
if you need to modify the elements of the line later on, using a list would be more suitable. If you want to ensure that the elements remain immutable, using a tuple would be appropriate.

In [4]:
window_size = 10
num_sessions = 0
inputs = []
outputs = []
with open('/content/hdfs_train', 'r') as f:
        for line in f:
            print(line)
            num_sessions += 1
            line = [int(n) - 1 for n in line.strip().split()]
            for i in range(len(line) - window_size):
                inputs.append(line[i:i + window_size])
                outputs.append(line[i + window_size])
            break

print(line)
print(inputs)
print(outputs)
print(num_sessions)

5 5 5 22 11 9 11 9 11 9 26 26 26 23 23 23 21 21 21 

[4, 4, 4, 21, 10, 8, 10, 8, 10, 8, 25, 25, 25, 22, 22, 22, 20, 20, 20]
[[4, 4, 4, 21, 10, 8, 10, 8, 10, 8], [4, 4, 21, 10, 8, 10, 8, 10, 8, 25], [4, 21, 10, 8, 10, 8, 10, 8, 25, 25], [21, 10, 8, 10, 8, 10, 8, 25, 25, 25], [10, 8, 10, 8, 10, 8, 25, 25, 25, 22], [8, 10, 8, 10, 8, 25, 25, 25, 22, 22], [10, 8, 10, 8, 25, 25, 25, 22, 22, 22], [8, 10, 8, 25, 25, 25, 22, 22, 22, 20], [10, 8, 25, 25, 25, 22, 22, 22, 20, 20]]
[25, 25, 25, 22, 22, 22, 20, 20, 20]
1


In [5]:
# what happen if the line is smaller than windows size or equal

line = '5 5 5 22 11 9 11'
window_size = 10
num_sessions = 0
inputs = []
outputs = []

line = [int(n) - 1 for n in line.strip().split()]
for i in range(len(line) - window_size):
    inputs.append(line[i:i + window_size])
    outputs.append(line[i + window_size])

print(line)
print(inputs)
print(outputs)


[4, 4, 4, 21, 10, 8, 10]
[]
[]


In [6]:
line = '5 5 5 22 11 9 11 9 11 9'
window_size = 10
num_sessions = 0
inputs = []
outputs = []

line = [int(n) - 1 for n in line.strip().split()]
for i in range(len(line) - window_size):
    inputs.append(line[i:i + window_size])
    outputs.append(line[i + window_size])

print(line)
print(inputs)
print(outputs)

[4, 4, 4, 21, 10, 8, 10, 8, 10, 8]
[]
[]


In [7]:
import torch
from torch.utils.data import TensorDataset

name = 'hdfs_train'
window_size = 10
num_sessions = 0
inputs = []
outputs = []

with open('/content/' + name, 'r') as f:
        for row in f:
            num_sessions += 1
            line = [int(i) - 1 for i in row.strip().split()] # we substract by one from templates index for starting from zero
            for i in range(len(line) - window_size):
                inputs.append(line[i:i + window_size])
                outputs.append(line[i + window_size])

print('Number of sessions({}): {}'.format(name, num_sessions))
print('Number of seqs({}): {}'.format(name, len(inputs)))
dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float), torch.tensor(outputs))


Number of sessions(hdfs_train): 4855
Number of seqs(hdfs_train): 46575


In [8]:
# ba carefull, we substract by one
inputs[0]

[4, 4, 4, 21, 10, 8, 10, 8, 10, 8]

In [9]:
outputs[0]

25

In [10]:
import numpy as np
print(np.max(outputs))
print(np.min(outputs))

25
1


In [11]:
# unique outputs
print(set(outputs))
len(set(outputs))

{1, 2, 3, 4, 5, 8, 10, 15, 17, 20, 21, 22, 24, 25}


14

In [12]:
dataset[0]

(tensor([ 4.,  4.,  4., 21., 10.,  8., 10.,  8., 10.,  8.]), tensor(25))

In [13]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(Model, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [14]:
input_size = 1 # means each input has one feature(template's code)
num_layers = 2
hidden_size = 64
num_classes = 28
batch_size = 2048
num_epochs = 300
model_dir = 'model'

In [15]:
log = 'Adam_batch_size={}_epoch={}'.format(str(batch_size), str(num_epochs))
writer = SummaryWriter(log_dir='log/' + log)


model = Model(input_size, hidden_size, num_layers, num_classes).to(device)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [16]:
for step, (seq, label) in enumerate(dataloader):
  print(seq.shape)
  print(label.shape)
  break

torch.Size([2048, 10])
torch.Size([2048])


In [17]:
# Train the model
start_time = time.time()
total_step = len(dataloader)
for epoch in range(num_epochs):  # Loop over the dataset multiple times
    train_loss = 0
    for step, (seq, label) in enumerate(dataloader):
        # Forward pass
        seq = seq.clone().detach().view(-1, window_size, input_size).to(device)
        output = model(seq)
        loss = criterion(output, label.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step))
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
print('Finished Training')

Epoch [1/300], train_loss: 2.8119
Epoch [2/300], train_loss: 1.8651
Epoch [3/300], train_loss: 1.7024
Epoch [4/300], train_loss: 1.5077
Epoch [5/300], train_loss: 1.2718
Epoch [6/300], train_loss: 1.0642
Epoch [7/300], train_loss: 0.9039
Epoch [8/300], train_loss: 0.7843
Epoch [9/300], train_loss: 0.6997
Epoch [10/300], train_loss: 0.6421
Epoch [11/300], train_loss: 0.6031
Epoch [12/300], train_loss: 0.5709
Epoch [13/300], train_loss: 0.5424
Epoch [14/300], train_loss: 0.5159
Epoch [15/300], train_loss: 0.4896
Epoch [16/300], train_loss: 0.4695
Epoch [17/300], train_loss: 0.4545
Epoch [18/300], train_loss: 0.4432
Epoch [19/300], train_loss: 0.4299
Epoch [20/300], train_loss: 0.4206
Epoch [21/300], train_loss: 0.4098
Epoch [22/300], train_loss: 0.4010
Epoch [23/300], train_loss: 0.3926
Epoch [24/300], train_loss: 0.3872
Epoch [25/300], train_loss: 0.3821
Epoch [26/300], train_loss: 0.3753
Epoch [27/300], train_loss: 0.3717
Epoch [28/300], train_loss: 0.3666
Epoch [29/300], train_loss: 0

In [18]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())


Model's state_dict:
lstm.weight_ih_l0 	 torch.Size([256, 1])
lstm.weight_hh_l0 	 torch.Size([256, 64])
lstm.bias_ih_l0 	 torch.Size([256])
lstm.bias_hh_l0 	 torch.Size([256])
lstm.weight_ih_l1 	 torch.Size([256, 64])
lstm.weight_hh_l1 	 torch.Size([256, 64])
lstm.bias_ih_l1 	 torch.Size([256])
lstm.bias_hh_l1 	 torch.Size([256])
fc.weight 	 torch.Size([28, 64])
fc.bias 	 torch.Size([28])


In [19]:
#save the model

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

torch.save(model.state_dict(), '/content/drive/MyDrive/model_parameter')


Mounted at /content/drive


In [20]:
x,y = dataset[0]
print(x)
print(y)

tensor([ 4.,  4.,  4., 21., 10.,  8., 10.,  8., 10.,  8.])
tensor(25)


In [21]:
model.eval()
with torch.no_grad():
  output = model(x.view(-1, 10, 1).to(device))

print(output)
print(np.argmax(output.cpu()))

tensor([[-4.7508,  1.8236, -0.0566,  1.1453, -2.2462, -5.7327, -3.7104, -4.8342,
          0.3210, -4.9804,  2.0385, -4.7240, -5.0413, -4.0583, -4.7007, -4.2532,
         -4.7936, -1.1776, -4.9065, -4.6588, -0.4992,  0.7799, -0.6379, -4.8290,
          1.5140, 12.9225, -4.7461, -4.4651]], device='cuda:0')
tensor(25)


In [22]:
x,y = dataset[5]
print(x)
print(y)

tensor([ 8., 10.,  8., 10.,  8., 25., 25., 25., 22., 22.])
tensor(22)


In [23]:
model.eval()
with torch.no_grad():
  output = model(x.view(-1, 10, 1).to(device))

print(output)
print(np.argmax(output.cpu()))

tensor([[-4.4647,  1.0807,  0.6209, -0.5563,  3.2914, -4.0438, -3.9144, -4.2281,
         -9.1934, -4.3735, -5.6101, -4.3301, -4.5008, -4.4556, -4.7793, -6.1979,
         -4.3460,  3.8124, -4.8607, -4.0699,  2.1028, -3.4791, 11.7600, -4.3012,
          1.8787, -4.2657, -4.2527, -4.2237]], device='cuda:0')
tensor(22)


# part2 : testing

In [24]:
def generate(name):
    # If you what to replicate the DeepLog paper results(Actually, I have a better result than DeepLog paper results),
    # you should use the 'list' not 'set' to obtain the full dataset, I use 'set' just for test and acceleration.
    hdfs = set()
    # hdfs = []
    with open('/content/' + name, 'r') as f:
        for row in f:
            line = [int(i) - 1 for i in row.strip().split()]
            line = line + [-1] * (window_size + 1 - len(line)) #if the length of the line is less than windows size, it covers by -1
            hdfs.add(tuple(line))
            # hdfs.append(tuple(line))
    print('Number of sessions({}): {}'.format(name, len(hdfs)))
    return hdfs

In [39]:
def generate2(name):
    # If you what to replicate the DeepLog paper results(Actually, I have a better result than DeepLog paper results),
    # you should use the 'list' not 'set' to obtain the full dataset, I use 'set' just for test and acceleration.
    #hdfs = set()
    hdfs = []
    with open('/content/' + name, 'r') as f:
        for row in f:
            line = [int(i) - 1 for i in row.strip().split()]
            line = line + [-1] * (window_size + 1 - len(line)) #if the length of the line is less than windows size, it covers by -1
            #hdfs.add(tuple(line))
            hdfs.append(tuple(line))
    print('Number of sessions({}): {}'.format(name, len(hdfs)))
    return hdfs

In [25]:
[-1] * 3

[-1, -1, -1]

In [26]:
[-1] * -2

[]

In [27]:
with open('/content/' + 'hdfs_test_abnormal', 'r') as f:
        for row in f:
            print(row)
            line = [int(i) - 1 for i in row.strip().split()]
            print(line)
            line = line + [-1] * (window_size + 1 - len(line)) #if the length of the line is less than windows size + 1(1 for prediction), it covers by -1
            print(line)
            break

5 5 22 5 11 9 11 9 11 9 26 26 26 4 4 3 2 23 23 23 21 21 28 26 21 

[4, 4, 21, 4, 10, 8, 10, 8, 10, 8, 25, 25, 25, 3, 3, 2, 1, 22, 22, 22, 20, 20, 27, 25, 20]
[4, 4, 21, 4, 10, 8, 10, 8, 10, 8, 25, 25, 25, 3, 3, 2, 1, 22, 22, 22, 20, 20, 27, 25, 20]


In [28]:
row = '5 5 22 5 11 9 11'
line = [int(i) - 1 for i in row.strip().split()]
print(line)
line = line + [-1] * (window_size + 1 - len(line)) #if the length of the line is less than windows size + 1(1 for prediction), it covers by -1
print(line)

[4, 4, 21, 4, 10, 8, 10]
[4, 4, 21, 4, 10, 8, 10, -1, -1, -1, -1]


In [29]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_keys):
        super(Model, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_keys)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [30]:
num_classes = 28
input_size = 1
num_candidates = 9 # on paper is g , top-g(here top 9) probabilities to appear next are considered normal
model = Model(input_size, hidden_size, num_layers, num_classes).to(device)

In [31]:
# upload the model

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

model_path = '/content/drive/MyDrive/model_parameter'
model.load_state_dict(torch.load(model_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<All keys matched successfully>

In [40]:
# if we use whole data , large part of it is duplicated
test_normal_loader = generate2('hdfs_test_normal')
test_abnormal_loader = generate2('hdfs_test_abnormal')

Number of sessions(hdfs_test_normal): 553366
Number of sessions(hdfs_test_abnormal): 16838


In [32]:
test_normal_loader = generate('hdfs_test_normal')
test_abnormal_loader = generate('hdfs_test_abnormal')

Number of sessions(hdfs_test_normal): 14177
Number of sessions(hdfs_test_abnormal): 4123


In [33]:
# Test the model

model.eval()

TP = 0
FP = 0
start_time = time.time()
with torch.no_grad():
    for line in test_normal_loader:
        for i in range(len(line) - window_size):
            seq = line[i:i + window_size]
            print(seq)
            label = line[i + window_size]
            print(label)
            seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
            print(seq)
            label = torch.tensor(label).view(-1).to(device)
            print(label)
            output = model(seq)

            predicted = torch.argsort(output, 1)[0][num_candidates:]
            break
        break

(4, 4, 4, 21, 10, 8, 25, 25, 10, 8)
10
tensor([[[ 4.],
         [ 4.],
         [ 4.],
         [21.],
         [10.],
         [ 8.],
         [25.],
         [25.],
         [10.],
         [ 8.]]], device='cuda:0')
tensor([10], device='cuda:0')


In [34]:
output

tensor([[-3.4265,  0.6027,  0.9818,  2.0122, -4.1019, -0.6238, -3.0547, -3.7285,
          4.8899, -3.5270, 12.8156, -3.1909, -3.2499, -3.0170, -3.1875, -2.2732,
         -3.5635, -3.4264, -3.5696, -3.2081,  1.6847,  0.3453, -2.4102, -3.3496,
         -1.0149,  2.7546, -2.9334, -3.3896]], device='cuda:0')

In [35]:
np.argmin(output.cpu())

tensor(4)

In [36]:
torch.argsort(output, 1)

tensor([[ 4,  7, 18, 16,  9,  0, 17, 27, 23, 12, 19, 11, 14,  6, 13, 26, 22, 15,
         24,  5, 21,  1,  2, 20,  3, 25,  8, 10]], device='cuda:0')

In [37]:
# Test the model
TP = 0
FP = 0

start_time = time.time()
with torch.no_grad():
    for line in test_normal_loader:
        for i in range(len(line) - window_size):
            seq = line[i:i + window_size]
            label = line[i + window_size]
            seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
            label = torch.tensor(label).view(-1).to(device)
            output = model(seq)
            predicted = torch.argsort(output, 1)[0][-num_candidates:]
            if label not in predicted:
                FP += 1
                break   #with just one wrong prediction in a line , we assume , abnormal
with torch.no_grad():
    for line in test_abnormal_loader:
        for i in range(len(line) - window_size):
            seq = line[i:i + window_size]
            label = line[i + window_size]
            seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
            label = torch.tensor(label).view(-1).to(device)
            output = model(seq)
            predicted = torch.argsort(output, 1)[0][-num_candidates:]
            if label not in predicted:
                TP += 1
                break
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
# Compute precision, recall and F1-measure
FN = len(test_abnormal_loader) - TP
P = 100 * TP / (TP + FP)
R = 100 * TP / (TP + FN)
F1 = 2 * P * R / (P + R)
print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
print('Finished Predicting')

elapsed_time: 157.939s
false positive (FP): 489, false negative (FN): 327, Precision: 88.588%, Recall: 92.069%, F1-measure: 90.295%
Finished Predicting


In [38]:
len(test_abnormal_loader)

4123